# Instalando bibliotecas

In [1]:
""" Installing libraries """
%pip install --quiet pandas==2.3.2 matplotlib==3.10.6 seaborn==0.13.2 scikit-learn==1.7.1 numpy==2.2.6 pyarrow==21.0.0
%pip install --quiet torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu129

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Importando bibliotecas (externas e próprias)

In [2]:
""" Importing libraries """

import sys
import os

# Add the parent directory to sys.path so 'Modules' can be imported
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Our modules
from Modules.evaluation.output_formatter import forecast_to_output
from Modules.loading.plug_n_play import get_clean_data
from Modules.loading.read_parquet import read_parquet_file 
from Modules.models.forecast import forecast_blind
from Modules.models.make_dataset import SingleSeriesDataset, MultiSeriesDataset
from Modules.models.NBeats import NBeatsBlock, NBeats
from Modules.models.test import hard_test
from Modules.models.test import soft_test
from Modules.models.training import train_model
from Modules.models.WMAPELoss import WMAPELoss
from Modules.preprocessing.onehot import one_hot_encode_parquet


# Definição dos hiper-parâmetros

In [3]:
""" Defining hyper-parameters """

HYPERPARAMS = {
    # Neural Network Global Parameters
    'input_size': 30,       # Number of past days to use as input
    'output_size': 7,       # Number of future days to predict
    'batch_size': 28,       # Batch size for training
    'n_layers': 4,          # Number of layers in the N-BEATS model
    'hidden_size': 128,     # Number of hidden units in each layer

    # Training parameters
    'learning_rate': 1e-3,  # Learning rate for the optimizer
    'epochs': 100,          # Number of training epochs (iterations over the entire dataset)
    'device': torch.device("cuda" if torch.cuda.is_available() else "cpu"),  # Use GPU if available
    'blind_horizon': 4,     # Number of days to exclude from the end of the training set for hard test
    'split': 1,             # Proportion of data to use for training (1.0 for validation)
    'seed': 42,             # Random seed for reproducibility
}

# Importação do Dataset

É interessante dividir o treino em batches (mini-conjuntos de treino). Cada batch possui o tamanho de input size, seguindo a ordem cronológica de vendas dentro daquela janela de dias. No entanto, durante o treinamento é **ESSENCIAL** que a escolha do próximo batch seja aleatória.

Ex.: Inicia o treino por 21-27 jul e prevê 28, depois pula para 02-08 fev para prever 03. Esse processo deve ser repetido até todos os dados serem treinados, finalizando **01 epoch**.

O número de **epochs** diz o número total de iterações do modelo com relação ao dataset inteiro.

Sobre a composição da janela de input dentro de um batch, existem duas abordagens:

1) Treinar em cada janela todas as séries (pense que cada par produto-loja x tempo representa uma série temporal dentro daquele período). Esse modelo é bem mais complexo pois o output deve ter o mesmo tamanho de produto-loja.
2) Treinar vários modelos separados (considerando uma série temporal para cada modelo). Esse método é ineficiente pois o modelo nunca irá aprender os padrões entre as séries.
3) Treinar o modelo com um par produto-loja por vez. Ou seja:
   - O modelo realiza epochs = N iterações de treino ao longo de todo dataset
     - Em cada epoch, passa por todas as M batches
       - Em cada batch (que possui uma janela de tamanho input_size), atualiza os parâmetros para cada série temporal ($x_l,y_l$). Totalizando L atualizações, com L sendo o número de pares produto-loja.

Ressalta-se que cada conjunto ($x_l,y_l$) representa:
- $x_l$: série temporal do l-ésimo par produto-loja, sendo um vetor de tamanho input_size x (features + 1)
- $y_l$: Previsão de vendas do l-ésimo par produto-loja para os próximos $output_size$ dias


In [4]:
""" Recebe os dados filtrados / limpos """

clean_data = get_clean_data(verbose=True)

00:00:202 FILE_PATHS
00:00:000 loaded_data
00:03:379 numerical_table
00:01:144 outlierless
00:01:138 pivoted_df
00:24:699 rescaled_df
00:17:918 copying
00:01:219 returning


In [14]:
""" get dataset function """

def get_dataset(sample, feature, hyperparams):
    input_size = hyperparams['input_size']
    output_size = hyperparams['output_size']

    # Inicializando os vetores de entrada e os rótulos
    X = []
    y = []

    # Número total de amostras (janelas diferentes) que podem ser extraídas
    # de um mesmo sample
    num_windows = len(sample) - input_size - output_size + 1
    print(f"Número total de janelas extraídas: {num_windows}")
    print(f"Para janelas de tamanho {input_size} e previsão de {output_size} dias à frente.")
    # Extraindo janelas deslizantes
    for i in range(num_windows):

        X_window = sample[i:i+input_size]             # janela de entrada
        feature_window = feature[i:i+input_size]      # janela de entrada

        X_window = np.stack([X_window,feature_window], axis=1)  # shape = (input_size, 4)

        y_window = sample[i+input_size:i+input_size+output_size]  # próximos dias da série

        X.append(X_window)
        y.append(y_window)

    # Convertendo para arrays numpy e depois para tensores PyTorch
    X = np.array(X)  # shape = [num_windows, input_size, num_features]
    y = np.array(y)  # shape = [num_windows, output_size]
    print()
    print("O vetor de entrada  antes do flatten")
    print(f"tem shape (num_windows, size_window, num_features): {X.shape}")

    X = torch.tensor(X, dtype=torch.float32)  # shape = [n_samples, input_size, 1]

    y = torch.tensor(y, dtype=torch.float32)  # shape = [n_samples, 1]


    dataset_full = SingleSeriesDataset(X, y)
    return dataset_full, X, y


In [15]:
""" Training Model """

def traininig_func(X_train, y_train, num_features, hyperparams):
    # Adotando o dataset de treino
    batch_size = hyperparams['batch_size']
    input_size = hyperparams['input_size']
    hidden_size = hyperparams['hidden_size']
    output_size = hyperparams['output_size']
    n_layers = hyperparams['n_layers']
    device = hyperparams['device']
    learning_rate = hyperparams['learning_rate']
    epochs = hyperparams['epochs']
    
    num_features = num_features
    
    dataset = SingleSeriesDataset(X_train, y_train) 
    dataloader = DataLoader(dataset, batch_size, shuffle=False) # shuffle=False para séries temporais

    # Inicialização do modelo N-BEATS (considerando X_train com num_features)
    model = NBeats(input_size*num_features, hidden_size, output_size, n_layers).to(device)
    model, criterion, optimizer = train_model(model, learning_rate, epochs, device, dataloader)
    return model


In [16]:
""" Training Model """

def get_blind_prediction(model, X_train, hyperparams):
    blind_horizon = hyperparams['blind_horizon']
    device = hyperparams['device']
    output_size = hyperparams['output_size']
    split = hyperparams['split']

    if split != 1.0:
        print("Previsão cega não realizada, pois split < 1.0")
        return

    blind_prediction = forecast_blind(model, X_train, blind_horizon, device, output_size)
    return blind_prediction
    


In [17]:
""" Separando treino e validação """

def get_train_validation(dataset_full, X, y, hyperparams):
    split = hyperparams['split']

    X_test, y_test = None, None
    X_train, y_train = None, None
    
    # Ponto de separação entre treino e validação (Caso seja para envio, não há validação)
    if split < 1:
        split_point = int(split * len(dataset_full))
        # Separação cronológica das janelas
        X_train, X_test = X[:split_point], X[split_point:]
        y_train, y_test = y[:split_point], y[split_point:]
    else:
        X_train = X
        y_train = y

    num_features = X_train.shape[2]
    
    # Flatten do tensor para entrar na rede
    X_train = X_train.view(X_train.shape[0], -1)  # shape = [num_windows_train, input_size * n_features]

    if split < 1:
        X_test  = X_test.view(X_test.shape[0], -1) # shape = [num_windows_test, input_size * n_features]

    print(f"Há um total de {len(dataset_full)} janelas e o split ocorre em {split*100:.0f}% do dataset")
    print(f" O shape de X_train é {X_train.shape} e o shape de X_test é {X_test.shape}") if split < 1 else ""
    print(f" O shape de y_train é {y_train.shape} e o shape de y_test é {y_test.shape}") if split < 1 else ""

    return X_train, y_train, X_test, y_test, num_features
    

In [18]:
""" Get Sample """

def get_sample(clean_data, col):
    sample = clean_data[col].values
    # sample = clean_data[sampled_col.sum()].values

    plt.figure(figsize=(12, 6))
    plt.bar(range(len(clean_data)), sample)
    return sample


In [19]:
""" Create Feature and Rescale """

def create_feature_rescale(sampled): # Rescaling e sampling
    sample = (sampled - np.min(sampled)) / (np.max(sampled) - np.min(sampled))
    
    feature = [sampled[t] - sampled[t-1] for t in range(1, len(sampled))]
    feature = (feature - np.min(feature)) / (np.max(feature) - np.min(feature))
    return sample, feature

In [20]:

def get_predictions(clean_data, prediction_col, hyperparams):
    sample = get_sample(clean_data, prediction_col)
    sample, feature = create_feature_rescale(sample)
    dataset_full, X, y = get_dataset(sample, feature, hyperparams)
    X_train, y_train, X_test, y_test, num_features = get_train_validation(dataset_full, X, y, hyperparams)
    model = traininig_func(X_train, y_train, num_features, hyperparams)
    blind_prediction = get_blind_prediction(model, X_train, hyperparams)
    return blind_prediction 


In [26]:
""" Get dataframe predictions """
def get_dataframe_predictions(clean_data, HYPERPARAMS):
    final_predictions = dict()

    for col in clean_data[:5]:
        final_predictions[col] = get_predictions(clean_data, col, HYPERPARAMS)
    return final_predictions

In [ ]:
""" Modularizando o código de predição """

final_predictions = get_dataframe_predictions(clean_data, HYPERPARAMS)


Número total de janelas extraídas: 329
Para janelas de tamanho 30 e previsão de 7 dias à frente.

O vetor de entrada  antes do flatten
tem shape (num_windows, size_window, num_features): (329, 30, 2)
Há um total de 329 janelas e o split ocorre em 100% do dataset
Epoch 1/100, Loss: 522392402.9838
Epoch 2/100, Loss: 213790258.2588
Epoch 3/100, Loss: 117154210.8618
Epoch 4/100, Loss: 86218116.8562
Epoch 5/100, Loss: 78751121.5168
Epoch 6/100, Loss: 111352649.8630
Epoch 7/100, Loss: 97748298.2090
Epoch 8/100, Loss: 94805119.1916
Epoch 9/100, Loss: 94329078.5347
Epoch 10/100, Loss: 92464550.5509
Epoch 11/100, Loss: 73285918.8484
Epoch 12/100, Loss: 82713816.8662
Epoch 13/100, Loss: 69639330.8535
Epoch 14/100, Loss: 81307077.1964
Epoch 15/100, Loss: 77465369.1967
Epoch 16/100, Loss: 68715701.1810
Epoch 17/100, Loss: 59515641.8523
Epoch 18/100, Loss: 43293903.8452
Epoch 19/100, Loss: 49760650.1875
Epoch 20/100, Loss: 45240857.3447
Epoch 21/100, Loss: 43133660.5101
Epoch 22/100, Loss: 54583632

In [ ]:
""" Output Formatting """

final_df = forecast_to_output(final_predictions, prediction_col)

# OLD

---

In [ ]:
""" Sampling and plotting - OK"""

sample = get_sample(clean_data, prediction_col)


# Transformando o Dataframe em um Dataset



In [ ]:
""" Rescaling e sampling - OK """

sample, feature = create_feature_rescale(sample)


In [ ]:
""" get dataset """

dataset_full, X, y = get_dataset(sample, feature, hyperparams)


# Separando os dados entre treino / validação

Os dados serão separados na proporção 80% - treino / 20% validação. Para séries temporais, é usual que essa separação seja feita de forma cronológica

In [ ]:
""" Separando treino e validação """

X_train, y_train, X_test, y_test, num_features = get_train_validation(dataset_full, X, y, hyperparams)


# Treinamento

In [ ]:
""" Training """

model = traininig_func(X_train, y_train, num_features, hyperparams)



# Sanity check do modelo treinado

In [ ]:
""" Sanity Check - TO-DO"""

# sanity_check_plot()

# ===

def sanity_check_plot(model, dataloader, device, output_size):
    """
    Plota previsões vs ground truth no período de treino
    """
    model.eval()
    all_preds, all_targets = [], []

    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)

            preds = model(x)
            
            # Se seu modelo retorna (batch, output_size)
            preds = preds.cpu().numpy().flatten()
            y = y.cpu().numpy().flatten()

            all_preds.extend(preds)
            all_targets.extend(y)

    # Plot
    plt.figure(figsize=(12, 5))
    plt.plot(all_targets, label="Ground truth", linewidth=2)
    plt.plot(all_preds, label="Previsões", linewidth=2, alpha=0.7)
    plt.title("Sanity check - Previsões vs Ground Truth (treino)")
    plt.legend()
    plt.show()

sanity_check_plot(model, dataloader, device, output_size)

In [ ]:
y

# Validação do modelo

São feitos dois testes:

- **Soft test:** Modelo  tenta fazer as previsões, mas não utiliza-as nas previsões futuras, utiliza sempre os *ground truth* como input
- **Hard test:** Modelo tenta fazer as previsões, e utiliza $y_{i-1}$ para a previsão de $y_i$

In [ ]:
""" Model Validation - TO-DO """

# validate_model()

# ===

" Validando o modelo quando o split é menor que 1"
if split < 1:
    # Adotando o dataset de validação (soft)
    dataset = SingleSeriesDataset(X_test, y_test)
    dataloader = DataLoader(dataset, batch_size, shuffle=False) # shuffle=False para séries temporais

    all_preds_S, all_targets_S, avg_loss_test_S = soft_test(model, dataloader, device, criterion)

    # Validação hard - previsão cega das primeiras blind_horizon semanas
    all_preds_H, all_targets_H, avg_loss_test_H = hard_test(model, X_train, y_train, y_test, split_point, device, criterion, blind_horizon, output_size)

    " Sanity check da validação do modelo (preds and targets)"
    all_preds_array = []
    all_targets_array = []

    # Convert lists to tensors before flattening
    all_preds_tensor = torch.cat([t.unsqueeze(0) if t.dim() == 1 else t for t in all_preds_H], dim=0).flatten()
    all_targets_tensor = torch.cat([t.unsqueeze(0) if t.dim() == 1 else t for t in all_targets_H], dim=0).flatten()

    for t in all_preds_tensor:
        all_preds_array.append(t.detach().numpy())
    for t in all_targets_tensor:
        all_targets_array.append(t.detach().numpy())


    plt.figure(figsize=(12, 5))
    plt.plot(all_targets_array, label="Ground truth", linewidth=2)
    plt.plot(all_preds_array, label="Previsões", linewidth=2, alpha=0.7)
    plt.legend()


# Previsão cega (quando split == 1)

Previsão para envio para o hackathon das 4 primeiras semanas de janeiro/23. Utiliza todo o dataset como treino.

In [ ]:
""" Blind prediction """

blind_prediction = get_blind_prediction(model, X_train, hyperparams)


# Ponto de parada - RAB 18-09-25
- Temporais (mais fáceis de pensar e implementar)
- Categóricas (intrínsecas do produto ou da loja, como a categoria deles)
- De localização (apenas se der tempo)

Após termos feito isso, podemos então realizar normalização dos sinais, one hot encoding do que for viável e embedding de IDs etc